In [ ]:
## Load the LabOne API and other necessary packages
from zhinst.toolkit import Session,  Waveforms
from zhinst.toolkit import CommandTable
import numpy as np

device_id='DEV9175'
server_host = 'localhost'

### connect to data server
session = Session(server_host)
### connect to device
device = session.connect_device(device_id)


# sequence parameters
nu = 244.1#385 #385 #100 #100#388#385.7 # microwave frequency in MHz
Fs = 2400 
NUM_SUBCYCLES = 100  # Number of subcycles #Number of points
tau_step0 =1.25E-3 # tau step in us
tau_start0 = 2.5E-3 # starting tau in us
t_tot = 78 # total time of a pulse frame in us 
TT_tot = t_tot*NUM_SUBCYCLES*2  # total time need for measuring all data points once
t_initi = 50 #laser initialization time
A = 0.65#0.5087 #0.6 #0.3393#0.3#0.7



#A = 0.36 # 100 T=35ns

# convert parameters
TAU_STEP = int(tau_step0*Fs) # convert tau step into tick
#TAU_START = int(tau_start0*Fs) # convert tau step into tick
DATA_POINTS_PER_SUBCYCLE = int(t_tot*Fs)  # Data points per subcycle
DATA_POINTS_TOTAL = DATA_POINTS_PER_SUBCYCLE*NUM_SUBCYCLES*2 # total waveform points for measuring all data points once
INITIAL_PERIOD_LENGTH =  int(tau_start0*Fs)  # Initial tau
MAX_PERIOD_LENGTH = INITIAL_PERIOD_LENGTH + (NUM_SUBCYCLES - 1) * TAU_STEP  # Maximum length of tau
WFM_LEN = NUM_SUBCYCLES * DATA_POINTS_PER_SUBCYCLE*2  # Total waveform length
T_INITIAL = int((t_initi+0.12)*Fs)


# define sequence veriable and send it the sequencer in LabOne

seqc_a = """\const LENGTH = """
seqc_b = str(DATA_POINTS_TOTAL)
seqc_c = """; // Number of subcycles * Data points per subcycle

wave w1 = placeholder(LENGTH, false, false); // Create a waveform of size LENGTH, with one marker
wave w2 = placeholder(LENGTH, false, false); // Create a waveform of size LENGTH, with one marker

assignWaveIndex(w1,w2,10);                 // Create a wave table entry with placeholder waveform              
                                         // routed to output 1, with index 10

// execute the command table entry
executeTableEntry(0);

while(true){
  waitDigTrigger(1);
  playWave(w1,w2);

  waitWave();
};
"""

seqc_program = seqc_a + seqc_b +seqc_c
# seqc_program = seqc_b
print(seqc_program)

AWG_INDEX = 0 #use channel 1&2 for 2*2
awg = device.awgs[AWG_INDEX]
WAVE_INDEX = 0
TABLE_INDEX = 0

# load the sequence
awg.load_sequencer_program(seqc_program)



  const LENGTH = 37440000; // Number of subcycles * Data points per subcycle

wave w1 = placeholder(LENGTH, false, false); // Create a waveform of size LENGTH, with one marker
wave w2 = placeholder(LENGTH, false, false); // Create a waveform of size LENGTH, with one marker

assignWaveIndex(w1,w2,10);                 // Create a wave table entry with placeholder waveform              
                                         // routed to output 1, with index 10

// execute the command table entry
executeTableEntry(0);

while(true){
  waitDigTrigger(1);
  playWave(w1,w2);

  waitWave();
};



{'messages': 'Warning (line: 16): broken or empty statement list detected\n',
 'maxelfsize': 2147483648}

In [2]:
# initialize the command table
# ct_schema = awg.commandtable.load_validation_schema()
# ct = CommandTable(ct_schema)

# Wavefrom eith amplitude and phase settings
# ct.table[TABLE_INDEX].waveform.index = WAVE_INDEX
# ct.table[TABLE_INDEX].amplitude0.value = 1
# ct.table[TABLE_INDEX].amplitude1.value = 1
#ct.table[TABLE_INDEX].phase0.value = 0
#ct.table[TABLE_INDEX].phase1.value = 90


wave1 = np.zeros(WFM_LEN)
wave2 = np.zeros(WFM_LEN)

# Generate sine waves for each subcycle
for subcycle in range(NUM_SUBCYCLES):
    current_length = INITIAL_PERIOD_LENGTH + subcycle * TAU_STEP  # Length for the current sine wave
    start_index = subcycle * DATA_POINTS_PER_SUBCYCLE*2 + T_INITIAL # Center position
    
    # Generate sine wave
    for i in range(current_length):
        wave1[start_index + i] = A*np.sin( 2*np.pi*i*nu/Fs)  # Sine wave calculation

# Generate cosine waves for each subcycle
for subcycle in range(NUM_SUBCYCLES):
    current_length = INITIAL_PERIOD_LENGTH + subcycle * TAU_STEP  # Length for the current sine wave
    start_index = subcycle * DATA_POINTS_PER_SUBCYCLE*2 + T_INITIAL# Center position
    
    # Generate sine wave
    for i in range(current_length):
        wave2[start_index + i] = A*np.cos( 2*np.pi*i*nu/Fs)  # Sine wave calculation

## Upload waveforms

waveforms = Waveforms()
waveforms[10] = (wave1,wave2,None) # I-component (output channel 1), Q-component (output channel 2), marker
# awg.commandtable.upload_to_device(ct)
awg.write_to_waveform_memory(waveforms)
awg.enable_sequencer(single=True)